In [1]:
import pandas as pd
import datetime
import time
from nba_api.stats.endpoints import LeagueDashPlayerStats
from nba_api.stats.static import players
import requests
from PIL import Image

In [2]:
# measure type: usage
# per mode: per game
# 2021-22 start date: October 19, 2021
# today: November 24, 2021
# count n games

# 2020-21 start date: December 22, 2020
# find second date for n games

In [3]:
TODAY = datetime.date.today()
CURR_SEASON_START = datetime.datetime.strptime('2021-10-19','%Y-%m-%d').date()
PAST_SEASON_START = datetime.datetime.strptime('2020-12-22','%Y-%m-%d').date()
PAST_SEASON_CHECKPOINT = PAST_SEASON_START + (TODAY - CURR_SEASON_START)

CURR_SEASON = '2021-22'
PAST_SEASON = '2020-21'

COLS = ['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'PCT_FTA']

In [4]:
CURR_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY
).get_data_frames()[0][COLS]

time.sleep(10)

In [5]:
PAST_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][COLS]

time.sleep(10)

In [6]:
CURR_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY,
    season=CURR_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]

time.sleep(10)

In [7]:
PAST_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]

time.sleep(10)

In [8]:
PLAYERS_LIST_REF = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    season=PAST_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]

In [9]:
PLAYERS_LIST_REF = PLAYERS_LIST_REF[
    (PLAYERS_LIST_REF['GP'] >= 41) &
    (PLAYERS_LIST_REF['MIN'] >= 1600)
].sort_values(by='FTA', ascending=False).iloc[:15]

In [10]:
PLAYERS_LIST = list(PLAYERS_LIST_REF['PLAYER_NAME'])

In [11]:
PAST_SEASON_FTA

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,FTA
0,Aaron Gordon,DEN,19,552.020000,5.2
1,Aaron Holiday,IND,23,465.575000,1.4
2,Aaron Nesmith,BOS,9,115.003333,0.0
3,Abdel Nader,PHX,9,159.605000,4.1
4,Adam Mokoka,CHI,6,41.115000,0.0
...,...,...,...,...,...
470,Yogi Ferrell,LAC,2,40.501667,1.8
471,Yuta Watanabe,TOR,16,199.366667,1.4
472,Zach LaVine,CHI,21,745.943333,5.4
473,Zeke Nnaji,DEN,10,23.361667,0.0


In [12]:
CURR_SEASON_STATS

# PLAYERS_LIST = list(CURR_SEASON_STATS['PLAYER_NAME'])

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
0,Aaron Gordon,DEN,21,673.0,0.250
1,Aaron Henry,PHI,1,2.0,0.000
2,Aaron Holiday,WAS,22,389.0,0.084
3,Aaron Nesmith,BOS,17,157.0,0.041
4,Aaron Wiggins,OKC,9,130.0,0.123
...,...,...,...,...,...
467,Yuta Watanabe,TOR,4,66.0,0.222
468,Yves Pons,MEM,1,3.0,0.000
469,Zach LaVine,CHI,23,799.0,0.322
470,Zeke Nnaji,DEN,9,109.0,0.333


In [13]:
PAST_SEASON_STATS

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
0,Aaron Gordon,DEN,19,552.0,0.319
1,Aaron Holiday,IND,23,466.0,0.088
2,Aaron Nesmith,BOS,9,115.0,0.000
3,Abdel Nader,PHX,9,160.0,0.247
4,Adam Mokoka,CHI,6,41.0,0.000
...,...,...,...,...,...
470,Yogi Ferrell,LAC,2,41.0,0.182
471,Yuta Watanabe,TOR,16,199.0,0.096
472,Zach LaVine,CHI,21,746.0,0.330
473,Zeke Nnaji,DEN,10,23.0,0.000


In [14]:
# combine data and begin graphing
CURR_SEASON_STATS.rename(columns={'PCT_FTA': 'CURR_FTR', 'MIN': 'CURR_MIN', 'GP': 'CURR_GP'}, inplace=True)
CURR_SEASON_STATS.drop(columns=['TEAM_ABBREVIATION'], inplace=True)

In [15]:
df = CURR_SEASON_STATS.merge(PAST_SEASON_STATS, left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df.rename(columns={'PCT_FTA': 'PAST_FTR', 'MIN': 'PAST_MIN', 'GP': 'PAST_GP'}, inplace=True)
df['FTR_DIFF'] = df['CURR_FTR'] - df['PAST_FTR']
df.sort_values(by='FTR_DIFF', ascending=False, inplace=True)

In [16]:
CURR_SEASON_FTA.rename(columns={'FTA': 'CURR_FTA'}, inplace=True)
PAST_SEASON_FTA.rename(columns={'FTA': 'PAST_FTA'}, inplace=True)

df = df.merge(CURR_SEASON_FTA[['PLAYER_NAME', 'CURR_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df = df.merge(PAST_SEASON_FTA[['PLAYER_NAME', 'PAST_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')

df['FTA_DIFF'] = df['CURR_FTA'] - df['PAST_FTA']


In [17]:
df.dropna(how='any', inplace=True)
df

,PLAYER_NAME,CURR_GP,CURR_MIN,CURR_FTR,TEAM_ABBREVIATION,PAST_GP,PAST_MIN,PAST_FTR,FTR_DIFF,CURR_FTA,PAST_FTA,FTA_DIFF
0,Vlatko Cancar,3,23.0,0.500,DEN,9,21.0,0.000,0.500,6.3,0.0,6.3
1,Sekou Doumbouya,2,16.0,0.571,DET,21,269.0,0.163,0.408,8.8,2.8,6.0
2,Zeke Nnaji,9,109.0,0.333,DEN,10,23.0,0.000,0.333,4.0,0.0,4.0
3,Tacko Fall,7,19.0,0.500,BOS,4,42.0,0.211,0.289,7.4,3.5,3.9
4,KZ Okpala,10,63.0,0.348,MIA,13,166.0,0.081,0.267,4.6,1.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...
367,Hamidou Diallo,16,233.0,0.153,DET,21,450.0,0.425,-0.272,2.8,7.0,-4.2
368,Udoka Azubuike,4,8.0,0.000,UTA,12,49.0,0.316,-0.316,0.0,4.4,-4.4
369,DeMarcus Cousins,1,15.0,0.000,LAC,18,325.0,0.321,-0.321,0.0,4.9,-4.9
370,Brodric Thomas,2,2.0,0.000,CLE,4,24.0,0.350,-0.350,0.0,10.4,-10.4


In [18]:
# export full dataset
df.to_csv('data/nba_ftr_diffs.csv')

# export past season's top 15 FTA
df.sort_values(by='PAST_FTA', ascending=False).iloc[:15].to_csv('data/nba_top15_fta.csv')

In [19]:
# Download top 15 player headshots
# Look up player ID list
# Save player IDs
PLAYER_IDS = dict()
IMG_STR = 'https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/202695.png'

ACTIVE_PLAYERS = players.get_active_players()

for player in ACTIVE_PLAYERS:
    if player['full_name'] in PLAYERS_LIST:
        PLAYER_IDS[player['full_name']] = player['id']

In [20]:

PLAYER_IDS

{'Giannis Antetokounmpo': 203507,
 'Bradley Beal': 203078,
 'Jimmy Butler': 202710,
 'Stephen Curry': 201939,
 'DeMar DeRozan': 201942,
 'Luka Doncic': 1629029,
 "De'Aaron Fox": 1628368,
 'Jerami Grant': 203924,
 'James Harden': 201935,
 'Damian Lillard': 203081,
 'Ja Morant': 1629630,
 'Collin Sexton': 1629012,
 'Karl-Anthony Towns': 1626157,
 'Zion Williamson': 1629627,
 'Trae Young': 1629027}